In [ ]:
import dotenv
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
dotenv.load_dotenv()

In [ ]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
  azure_endpoint = "https://gpteel.openai.azure.com/", 
  model="ideta-gpt-3-5-turbo",
  api_version="2024-02-15-preview"
)


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("insea-actuariat.pdf")
docs = loader.load()

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
splits = text_splitter.split_documents(docs)


In [ ]:
from langchain_community.vectorstores import FAISS

from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(azure_endpoint = "https://gpteel.openai.azure.com/",
                               model="ada_embedding", api_key = "11abcd38b05146fe84a0639993c6dd49" , 
                               openai_api_version = "2024-02-15-preview", openai_api_type = "azure")


In [ ]:
vectorstore = FAISS.from_documents(splits, embeddings)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt)

In [ ]:
from langchain_core.messages import HumanMessage

chat_history = []

question = "quelles sont les cours de actuariat?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])

second_question = "tu peux le dire en anglais?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])